In [15]:
import yaml
import lhsmdu
import numpy as np
from scipy.stats import distributions

In [16]:
def make_configs(params, conditions, *, N):
    A = lhsmdu.sample(len(params), N)
    distros = {
        k: distributions.randint(0, len(p_vals))
        for k, p_vals in params.items()
    }
    idx_map = {
        key: np.int32(lhsmdu.inverseTransformSample(distros[key], floats)).flatten().tolist()
        for floats, key in zip(A, params)
    }

    configs = []
    for b in zip(*idx_map.values()):
        d = {k: params[j] for j, (k, params) in zip(b, params.items())}
        if conditions(d):
            configs.append(d)
    return configs

In [17]:
noalias_dumper = yaml.dumper.SafeDumper
noalias_dumper.ignore_aliases = lambda self, data: True
def dump_configs(configs, filename):
    with open(filename, "w") as f:
        f.write(
            "# Copyright (c) 2020, XMOS Ltd, All rights reserved\n"
            "# RANDOMLY GENERATED CONFIGS, MODIFY AT OWN RISK\n"
        )
        yaml.dump({"default": dict(enumerate(configs))}, stream=f, Dumper=noalias_dumper)

# test_conv2d.yml

In [18]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,3,4,5,7,9],
    input_channels=[4,8,12,16,20,32,36,48],
    output_channels=[4,8,12,16,28,32,48,52,64],
    padding=["same", "valid"],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [19]:
def conditions(d):
    if (d["K_w"] == d["K_h"] == 1 or d["K_w"] * d["input_channels"] <= 32):
        return False
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["padding"] == "valid":
        if d["K_w"] == d["width"] and d["K_h"] == d["height"]:
            # this would be single pixel conv2d
            return False
        return d["K_w"] <= d["width"] and d["K_h"] <= d["height"]
    return True

In [20]:
configs = make_configs(params, conditions, N=100)
len(configs)

45

In [21]:
dump_configs(configs, "test_conv2d.yml")

# test_custom_relu_conv2d.yml

In [22]:
params["max_value"] = [j/2 for j in range(1, 12)]

In [23]:
configs = make_configs(params, conditions, N=50)
len(configs)

19

In [24]:
dump_configs(configs, "test_custom_relu_conv2d.yml")

# test_padded_conv2d.yml

In [25]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,3,4,5,7,9],
    input_channels=[4,8,12,16,20,32,36,48],
    output_channels=[4,8,12,16,28,32,48,52,64],
    pad_t=[0,1,2],
    pad_b=[0,1,2],
    pad_l=[0,1,2],
    pad_r=[0,1,2],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [26]:
def conditions(d):
    if (d["K_w"] == d["K_h"] == 1 or d["K_w"] * d["input_channels"] <= 32):
        return False
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["pad_t"] == d["pad_b"] == d["pad_l"] == d["pad_r"] == 0:
        return False
    padded_width = d["width"] + d["pad_l"] + d["pad_r"]
    padded_height = d["height"] + d["pad_t"] + d["pad_b"]
    if d["K_w"] == padded_width and d["K_h"] == padded_height:
        # this would be single pixel conv2d
        return False
    return (
        d["K_w"] <= padded_width
        and d["K_h"] <= padded_height
        and d["pad_r"] < d["K_w"] > d["pad_l"]
        and d["pad_t"] < d["K_h"] > d["pad_b"]
    )    

In [27]:
configs = make_configs(params, conditions, N=70)
len(configs)

16

In [28]:
dump_configs(configs, "test_padded_conv2d.yml")

# test_conv2d_1x1.yml

In [29]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    input_channels=[4,8,12,16,20,32,36,48],
    output_channels=[4,8,12,16,28,32,48,52,64],
    num_threads=[1,2,5],
)

In [30]:
def conditions(d):
    if d["width"] == d["height"] == 1:
        # this would be single pixel conv2d
        return False
    return not (d["num_threads"] > d["height"] or d["num_threads"] > d["width"])

In [31]:
configs = make_configs(params, conditions, N=55)
len(configs)

37

In [32]:
dump_configs(configs, "test_conv2d_1x1.yml")

# test_conv2d_shallowin.yml

In [33]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,2,3,4,5,7,8],
    input_channels=[4,8,16,32],
    output_channels=[4,8,12,16,28,32,48,52,64],
    padding=["same", "valid"],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [34]:
def conditions(d):
    if (d["K_w"] == d["K_h"] == 1 or d["K_w"] * d["input_channels"] > 32):
        return False
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["padding"] == "valid":
        if d["K_w"] == d["width"] and d["K_h"] == d["height"]:
            # this would be single pixel conv2d
            return False
        return d["K_w"] <= d["width"] and d["K_h"] <= d["height"]
    return True

In [35]:
configs = make_configs(params, conditions, N=150)
len(configs)

52

In [36]:
dump_configs(configs, "test_conv2d_shallowin.yml")

# test_padded_conv2d_shallowin.yml

In [37]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,2,3,4,5,7,8],
    input_channels=[4,8,16,32],
    output_channels=[4,8,12,16,28,32,48,52,64],
    pad_t=[0,1,2],
    pad_b=[0,1,2],
    pad_l=[0,1,2],
    pad_r=[0,1,2],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [38]:
def conditions(d):
    if (d["K_w"] == d["K_h"] == 1 or d["K_w"] * d["input_channels"] > 32):
        return False
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["pad_t"] == d["pad_b"] == d["pad_l"] == d["pad_r"] == 0:
        return False
    padded_width = d["width"] + d["pad_l"] + d["pad_r"]
    padded_height = d["height"] + d["pad_t"] + d["pad_b"]
    if d["K_w"] == padded_width and d["K_h"] == padded_height:
        # this would be single pixel conv2d
        return False
    return (
        d["K_w"] <= padded_width
        and d["K_h"] <= padded_height
        and d["pad_r"] < d["K_w"] > d["pad_l"]
        and d["pad_t"] < d["K_h"] > d["pad_b"]
    )   

In [39]:
configs = make_configs(params, conditions, N=150)
len(configs)

20

In [40]:
dump_configs(configs, "test_padded_conv2d_shallowin.yml")

# test_depthwise_conv2d.yml

In [41]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,2,3,4,5,7,8],
    channels=[4,8,12,16,28,32,48,52,64],
    padding=["same", "valid"],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [42]:
def conditions(d):
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["padding"] == "valid":
        return d["K_w"] <= d["width"] and d["K_h"] <= d["height"]
    return True

In [43]:
configs = make_configs(params, conditions, N=70)
len(configs)

41

In [44]:
dump_configs(configs, "test_depthwise_conv2d.yml")

# test_padded_depthwise_conv2d.yml

In [45]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3,5,7,9],
    K_w=[1,2,3,4,5,7,8],
    channels=[4,8,12,16,28,32,48,52,64],
    pad_t=[0,1,2],
    pad_b=[0,1,2],
    pad_l=[0,1,2],
    pad_r=[0,1,2],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [46]:
def conditions(d):
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    if d["pad_t"] == d["pad_b"] == d["pad_l"] == d["pad_r"] == 0:
        return False
    return (
        d["K_w"] <= d["width"] + d["pad_l"] + d["pad_r"]
        and d["K_h"] <= d["height"] + d["pad_t"] + d["pad_b"]
        and d["pad_r"] < d["K_w"] > d["pad_l"]
        and d["pad_t"] < d["K_h"] > d["pad_b"]
    )

In [47]:
configs = make_configs(params, conditions, N=70)
len(configs)

20

In [48]:
dump_configs(configs, "test_padded_depthwise_conv2d.yml")

# test_maxpool2d.yml

In [49]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    K_h=[1,2,3],
    K_w=[1,2,3],
    channels=[4,8,12,16,28,32,48,52,64],
    padding=["valid"],
    strides=[(1,1), (1,2), (2,1), (2,2), (1, 3), (3, 2)],
    num_threads=[1,2,5],
)

In [50]:
def conditions(d):
    if d["num_threads"] > d["height"] or d["num_threads"] > d["width"]:
        return False
    return d["K_w"] <= d["width"] and d["K_h"] <= d["height"]

In [51]:
configs = make_configs(params, conditions, N=62)
len(configs)

39

In [52]:
dump_configs(configs, "test_maxpool2d.yml")

# test_avgpool2d.yml

In [53]:
configs = make_configs(params, conditions, N=62)
len(configs)

40

In [54]:
dump_configs(configs, "test_avgpool2d.yml")

# test_global_avgpool2d.yml

In [55]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    channels=[4,8,12,16,28,32,48,52,64],
    num_threads=[1,2,5],
)

In [56]:
def conditions(d):
    return not (
        d["num_threads"] > d["height"] or d["num_threads"] > d["width"]
    )

In [57]:
configs = make_configs(params, conditions, N=55)
len(configs)

38

In [58]:
dump_configs(configs, "test_global_avgpool2d.yml")

# Activations

In [59]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    channels=[1,4,7,12,16,25,32,49],
    num_threads=[1,2,5],
)

In [60]:
for file in ["test_relu.yml", "test_relu6.yml", "test_sigmoid.yml", "test_tanh.yml"]:
    configs = make_configs(params, conditions=lambda _: True, N=40)
    print(file, len(configs))
    dump_configs(configs, file)

test_relu.yml 40
test_relu6.yml 40
test_sigmoid.yml 40
test_tanh.yml 40


# test_fully_connected.yml

In [61]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    channels=[1,4,7,12,16,25,32,49],
    outputs=[1,4,7,12,16,25,32,49],
    num_threads=[1,2,5],
)

In [62]:
configs = make_configs(params, conditions=lambda _: True, N=40)
len(configs)

40

In [63]:
dump_configs(configs, "test_fully_connected.yml")

# test_single_pixel_conv2d.yml

In [64]:
params = dict(
    K_h=[1,2,4,5,8,12,15],
    K_w=[1,3,4,7,9,14,20],
    input_channels=[1,4,7,12,16,25,32,49],
    output_channels=[1,4,7,12,16,25,32,49],
    num_threads=[1,2,5],
)

In [65]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [66]:
dump_configs(configs, "test_single_pixel_conv2d.yml")

# test_bsign.yml

In [67]:
params = dict(
    height=[1,4,5,8],
    width=[1,4,7,8],
    input_channels=[i*32 for i in (1,4,7)],
    num_threads=[1,2,5],
)

In [68]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [69]:
dump_configs(configs, "test_bsign.yml")

# test_bconv2d_bin_DI.yml

In [70]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[256,512],
    output_channels=[32, 64],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [71]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [72]:
dump_configs(configs, "test_bconv2d_bin_DI.yml")

# test_bconv2d_bin_DI_padded.yml

In [73]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[2,3,6],
    K_w=[3,4,5],
    input_channels=[256,512],
    output_channels=[32, 64],
    padding=["same"],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [74]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [75]:
dump_configs(configs, "test_bconv2d_bin_DI_padded.yml")

# test_bconv2d_bin.yml

In [76]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[32, 128],
    output_channels=[32, 64],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [77]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [78]:
dump_configs(configs, "test_bconv2d_bin.yml")

# test_bconv2d_bin_padded.yml

In [79]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[32, 128],
    output_channels=[32, 64],
    padding=["same"],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    num_threads=[1,2,5],
)

In [80]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [81]:
dump_configs(configs, "test_bconv2d_bin_padded.yml")

# test_pad.yml

In [82]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[1,4,7,8,14,20],
    channels=[4,8,12,16,28,32,48,52,64],
    pad_t=[0,1,2],
    pad_b=[0,1,2],
    pad_l=[0,1,2],
    pad_r=[0,1,2],
)

In [83]:
def conditions(d):
    return sum(params["pad_" + k] for k in "tblr")

In [84]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [85]:
dump_configs(configs, "test_pad.yml")

# test_bconv2d_int8_DIDO.yml

In [86]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[256,512],
    output_channels=[16, 48],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    output_range = [(range_min, range_max) for range_min in range(-4, 1, 2) for range_max in range(1, 6, 2)],
    num_threads=[1,2,5],
)

In [87]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [88]:
dump_configs(configs, "test_bconv2d_int8_DIDO.yml")

# test_bconv2d_int8.yml

In [89]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[32, 128, 256+64],
    output_channels=[4, 28, 32],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    output_range = [(range_min, range_max) for range_min in range(-4, 1, 2) for range_max in range(1, 6, 2)],
    num_threads=[1,2,5],
)

In [90]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [91]:
dump_configs(configs, "test_bconv2d_int8.yml")

# test_bconv2d_int8_DIDO_padded.yml

In [92]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[256,512],
    output_channels=[16, 48],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    padding=["same"],
    output_range = [(range_min, range_max) for range_min in range(-4, 1, 2) for range_max in range(1, 6, 2)]
#     num_threads=[1,2,5],
)

In [93]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [94]:
dump_configs(configs, "test_bconv2d_int8_DIDO_padded.yml")

# test_bconv2d_int8_padded.yml

In [95]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[32, 128, 256+64],
    output_channels=[4, 28, 32],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    padding=["same"],
    output_range = [(range_min, range_max) for range_min in range(-4, 1, 2) for range_max in range(1, 6, 2)],
    num_threads=[1,2,5],
)

In [96]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [97]:
dump_configs(configs, "test_bconv2d_int8_padded.yml")

# test_bconv2d_int8_activation.yml

In [98]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[32, 128, 256+64],
    output_channels=[4, 28, 32],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    output_range = [(range_min, range_max) for range_min in range(-4, 1, 2) for range_max in range(1, 6, 2)],
    activation = ["relu", "relu6"],
    num_threads=[1,2,5],
)

In [99]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [100]:
dump_configs(configs, "test_bconv2d_int8_activation.yml")

# test_bconv2d_int8_DIDO_activation.yml

In [101]:
params = dict(
    height=[7, 10, 12],
    width=[6, 8, 11],
    K_h=[1,2,3,6],
    K_w=[1,3,4,5],
    input_channels=[256,512],
    output_channels=[16, 48],
    strides=[(1,1), (1,2), (2,1), (2,2)],
    output_range = [(range_min, range_max) for range_min in range(-4, 1, 2) for range_max in range(1, 6, 2)],
    activation = ["relu", "relu6"],
    num_threads=[1,2,5],
)

In [102]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [103]:
dump_configs(configs, "test_bconv2d_int8_DIDO_activation.yml")

# test_add.yml

In [104]:
params = dict(
    height=[1,3,5,8,13,15],
    width=[1,5,6,7,9,11],
    channels=[4,11,13,51,53,64],
    num_threads=[1,2,5],
)

In [105]:
configs = make_configs(params, conditions=lambda _: True, N=20)
len(configs)

20

In [106]:
dump_configs(configs, "test_add.yml")

In [107]:
# test_strided_slice.yml

In [111]:
params = dict(
    height=[1,4,5,8,12,15],
    width=[4,7,8,14,20],
    channels=[4,8,12,16,20,32,36,48],
)

In [112]:
def conditions(d):
    return True

In [113]:
configs = make_configs(params, conditions, N=100)
len(configs)

100

In [ ]:
dump_configs(configs, "test_strided_slice.yml")